<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit8/unit8_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="In Colab öffnen"/></a>

# Unit 8 Teil 2: Fortgeschrittenes Deep Reinforcement Learning. Verwendung von Sample Factory zur Wiedergabe von Doom aus Pixeln

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail2.png" alt="Thumbnail"/>

In diesem Notizbuch lernen wir, wie man ein Deep Neural Network trainiert, um Objekte in einer 3D-Umgebung zu sammeln, die auf dem Spiel Doom basiert; ein Video der resultierenden Policy ist unten zu sehen. Wir trainieren diese Strategie mit [Sample Factory] (https://www.samplefactory.dev/), einer asynchronen Implementierung des PPO-Algorithmus.

Bitte beachten Sie die folgenden Punkte:

* [Sample Factory](https://www.samplefactory.dev/) ist ein fortgeschrittenes RL-Framework und funktioniert **nur unter Linux und Mac** (nicht unter Windows).

* Das Framework funktioniert am besten auf einer **GPU-Maschine mit vielen CPU-Kernen**, wo es Geschwindigkeiten von 100k Interaktionen pro Sekunde erreichen kann. Die auf einem Standard-Colab-Notebook verfügbaren Ressourcen **begrenzen die Leistung dieser Bibliothek**. Daher spiegelt die Geschwindigkeit in dieser Einstellung **nicht die reale Leistung wider**.
* Benchmarks für Sample Factory sind in einer Reihe von Einstellungen verfügbar, schauen Sie sich die [examples](https://github.com/alex-petrenko/sample-factory/tree/master/sf_examples) an, wenn Sie mehr erfahren möchten.


In [ ]:
from IPython.display import HTML

HTML('''<video width="640" height="480" controls>
  <source src="https://huggingface.co/edbeeching/doom_health_gathering_supreme_3333/resolve/main/replay.mp4"
  type="video/mp4">Your browser does not support the video tag.</video>'''
)

Um dieses Hands-on für den [Zertifizierungsprozess] (https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process) zu validieren, müssen Sie ein Modell schieben:

- `doom_health_gathering_supreme` ein Ergebnis von >= 5 erhalten.

Um Ihr Ergebnis zu ermitteln, gehen Sie zur [Bestenliste] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) und suchen Sie Ihr Modell, **das Ergebnis = mean_reward - std of reward**

Wenn Sie Ihr Modell nicht finden, **gehen Sie zum Ende der Seite und klicken Sie auf die Schaltfläche "Aktualisieren "**.

Weitere Informationen über den Zertifizierungsprozess finden Sie in diesem Abschnitt 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Die GPU einstellen 💪.
- Um **das Training des Agenten zu beschleunigen, werden wir einen Grafikprozessor** verwenden. Gehen Sie dazu auf "Runtime > Change Runtime type".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Schritt 1">

- Hardware-Beschleuniger > GPU".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Schritt 2">

Bevor wir mit dem Training unseres Agenten beginnen, sollten wir **die Bibliothek und die Umgebungen studieren, die wir verwenden werden**.

## Beispiel Fabrik

[Sample Factory] (https://www.samplefactory.dev/) ist eine der **schnellsten RL-Bibliotheken, die sich auf sehr effiziente synchrone und asynchrone Implementierungen von Policy-Gradienten (PPO)** konzentriert.

Sample Factory ist gründlich **getestet, wird von vielen Forschern und Praktikern** verwendet und wird aktiv gepflegt. Unsere Implementierung ist bekannt dafür, dass sie **in einer Vielzahl von Domänen SOTA-Leistung erreicht und gleichzeitig die Trainingszeit für RL-Experimente und die Hardwareanforderungen minimiert**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactoryenvs.png" alt="Musterfabrik"/>



### Hauptmerkmale

- Hochgradig optimierte Algorithmus-[Architektur](https://www.samplefactory.dev/06-architecture/overview/) für maximalen Lerndurchsatz
- Synchrone und asynchrone](https://www.samplefactory.dev/07-advanced-topics/sync-async/) Trainingsregime
- Serieller (Einzelprozess-)Modus](https://www.samplefactory.dev/07-advanced-topics/serial-mode/) für einfaches Debugging
- Optimale Leistung sowohl in CPU-basierten als auch [GPU-beschleunigten Umgebungen](https://www.samplefactory.dev/09-environment-integrations/isaacgym/)
- Einzel- und Multi-Agenten-Training, Self-Play, unterstützt [Training mehrerer Policies](https://www.samplefactory.dev/07-advanced-topics/multi-policy-training/) gleichzeitig auf einem oder mehreren GPUs
- Populationsbasiertes Training ([PBT](https://www.samplefactory.dev/07-advanced-topics/pbt/))
- Diskrete, kontinuierliche, hybride Aktionsräume
- Vektorbasierte, bildbasierte, wörterbuchbasierte Beobachtungsräume
- Erstellt automatisch eine Modellarchitektur durch Analyse der Spezifikation des Aktions-/Beobachtungsraums. Unterstützt [benutzerdefinierte Modellarchitekturen](https://www.samplefactory.dev/03-customization/custom-models/)
- Entwickelt, um in andere Projekte importiert zu werden, sind [benutzerdefinierte Umgebungen](https://www.samplefactory.dev/03-customization/custom-environments/) Bürger erster Klasse
- Detaillierte [WandB und Tensorboard Zusammenfassungen](https://www.samplefactory.dev/05-monitoring/metrics-reference/), [benutzerdefinierte Metriken](https://www.samplefactory.dev/05-monitoring/custom-metrics/)
- [HuggingFace 🤗 Integration](https://www.samplefactory.dev/10-huggingface/huggingface/) (trainierte Modelle und Metriken in den Hub hochladen)
- [Mehrere](https://www.samplefactory.dev/09-environment-integrations/mujoco/) [Beispiel](https://www.samplefactory.dev/09-environment-integrations/atari/) [Umgebung](https://www.samplefactory.dev/09-environment-integrations/vizdoom/) [Integrationen](https://www.samplefactory.dev/09-environment-integrations/dmlab/) mit abgestimmten Parametern und trainierten Modellen

Alle oben genannten Richtlinien sind auf dem 🤗 Hub verfügbar. Suchen Sie nach dem Tag [sample-factory](https://huggingface.co/models?library=sample-factory&sort=downloads)

### Wie sample-factory funktioniert

Sample-factory ist eine der **höchst optimierten RL-Implementierungen, die der Community zur Verfügung stehen**.

Sie funktioniert, indem sie **mehrere Prozesse startet, die Rollout-Worker, Inferenz-Worker und einen Learner-Worker** ausführen.

Die *Arbeiter* **kommunizieren über gemeinsamen Speicher, was die Kommunikationskosten zwischen den Prozessen senkt**.

Die *Rollout Worker* interagieren mit der Umgebung und senden Beobachtungen an die *Inference Worker*.

Die *Inferenz-Worker* fragen eine feste Version der Richtlinie ab und **senden Aktionen zurück an den Rollout-Worker**.

Nach *k* Schritten senden die Rollout-Worker eine Trajektorie der Erfahrungen an den Learner-Worker, **die dieser zur Aktualisierung des Policy-Netzwerks des Agenten verwendet**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactory.png" alt="Beispielfabrik"/>

### Actor Critic Modelle in der Sample-Fabrik

Actor Critic-Modelle in Sample Factory bestehen aus drei Komponenten:

- **Encoder** - Verarbeitet Eingangsbeobachtungen (Bilder, Vektoren) und bildet sie auf einen Vektor ab. Dies ist der Teil des Modells, den Sie am ehesten anpassen möchten.
- **Kern** - Integriert Vektoren von einem oder mehreren Encodern, kann optional eine ein- oder mehrschichtige LSTM/GRU in einem speicherbasierten Agenten enthalten.
- **Decoder** - Wenden Sie zusätzliche Schichten auf die Ausgabe des Modellkerns an, bevor Sie die Richtlinien- und Wertausgaben berechnen.

Die Bibliothek wurde so konzipiert, dass sie automatisch beliebige Beobachtungs- und Aktionsräume unterstützt. Benutzer können ihre eigenen Modelle leicht hinzufügen. Weitere Informationen finden Sie in der [Dokumentation] (https://www.samplefactory.dev/03-customization/custom-models/#actor-critic-models-in-sample-factory).

## ViZDoom

[ViZDoom](https://vizdoom.cs.put.edu.pl/) ist eine **Open-Source-Python-Schnittstelle für die Doom Engine**.

Die Bibliothek wurde 2016 von Marek Wydmuch und Michal Kempka am Institut für Informatik der Technischen Universität Poznan, Polen, entwickelt.

Die Bibliothek ermöglicht das **Training von Agenten direkt von den Bildschirmpixeln in einer Reihe von Szenarien**, einschließlich Team-Deathmatch, wie im Video unten gezeigt. Da die ViZDoom-Umgebung auf einem Spiel basiert, das in den 90er Jahren entwickelt wurde, kann sie auf moderner Hardware mit beschleunigter Geschwindigkeit ausgeführt werden, **was es uns ermöglicht, komplexe KI-Verhaltensweisen ziemlich schnell zu erlernen**.

Die Bibliothek umfasst Funktionen wie:

- Multiplattform (Linux, macOS, Windows),
- API für Python und C++,
- [OpenAI Gym](https://www.gymlibrary.dev/) Umgebungs-Wrapper
- Leicht zu erstellende benutzerdefinierte Szenarien (visuelle Editoren, Skriptsprache und Beispiele verfügbar),
- Asynchroner und synchroner Einzel- und Mehrspielermodus,
- Leichtgewichtig (wenige MB) und schnell (bis zu 7000 fps im Sync-Modus, single-threaded),
- Anpassbare Auflösung und Rendering-Parameter,
- Zugriff auf den Tiefenpuffer (3D-Vision),
- Automatische Beschriftung der im Bild sichtbaren Spielobjekte,
- Zugriff auf den Audiopuffer
- Zugriff auf die Liste der Akteure/Objekte und die Kartengeometrie,
- Off-Screen-Rendering und Episodenaufzeichnung,
- Zeitskalierung im asynchronen Modus.

## Zuerst müssen wir einige Abhängigkeiten installieren, die für die ViZDoom-Umgebung erforderlich sind

Nachdem unsere Colab-Laufzeitumgebung nun eingerichtet ist, können wir mit der Installation der Abhängigkeiten beginnen, die für die Ausführung von ViZDoom unter Linux erforderlich sind.

Wenn Sie auf Ihrem Mac arbeiten, folgen Sie bitte den Installationsanweisungen auf der [github-Seite] (https://github.com/Farama-Foundation/ViZDoom/blob/master/doc/Quickstart.md#-quickstart-for-macos-and-anaconda3-python-36).

In [ ]:
%%capture
%%bash
# Install ViZDoom deps from
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip ffmpeg

# Boost libraries
apt-get install libboost-all-dev

# Lua binding dependencies
apt-get install liblua5.1-dev

## Dann können wir Sample Factory und ViZDoom installieren.
- Dies kann 7 Minuten dauern

In [ ]:
# install python libraries
# thanks toinsson
!pip install faster-fifo==1.4.2
!pip install vizdoom

In [ ]:
!pip install sample-factory==2.0.2

## Einrichten der Doom-Umgebung in sample-factory

In [ ]:
import functools

from sample_factory.algo.utils.context import global_model_factory
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl

from sf_examples.vizdoom.doom.doom_model import make_vizdoom_encoder
from sf_examples.vizdoom.doom.doom_params import add_doom_env_args, doom_override_defaults
from sf_examples.vizdoom.doom.doom_utils import DOOM_ENVS, make_doom_env_from_spec


# Registers all the ViZDoom environments
def register_vizdoom_envs():
    for env_spec in DOOM_ENVS:
        make_env_func = functools.partial(make_doom_env_from_spec, env_spec)
        register_env(env_spec.name, make_env_func)

# Sample Factory allows the registration of a custom Neural Network architecture
# See https://github.com/alex-petrenko/sample-factory/blob/master/sf_examples/vizdoom/doom/doom_model.py for more details
def register_vizdoom_models():
    global_model_factory().register_encoder_factory(make_vizdoom_encoder)


def register_vizdoom_components():
    register_vizdoom_envs()
    register_vizdoom_models()

# parse the command line args and create a config
def parse_vizdoom_cfg(argv=None, evaluation=False):
    parser, _ = parse_sf_args(argv=argv, evaluation=evaluation)
    # parameters specific to Doom envs
    add_doom_env_args(parser)
    # override Doom default values for algo parameters
    doom_override_defaults(parser)
    # second parsing pass yields the final configuration
    final_cfg = parse_full_cfg(parser, argv)
    return final_cfg

Nun, da die Einrichtung abgeschlossen ist, können wir den Agenten trainieren. Wir haben uns hier für eine ViZDoom-Aufgabe namens "Health Gathering Supreme" entschieden.

### Das Szenario: Health Gathering Supreme

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/Health-Gathering-Supreme.png" alt="Health-Gathering-Supreme"/>



Das Ziel dieses Szenarios ist es, **dem Agenten beizubringen, wie er überleben kann, ohne zu wissen, was ihn überleben lässt**. Der Agent weiß nur, dass **das Leben kostbar** und der Tod schlecht ist, also **muss er lernen, was seine Existenz verlängert und dass seine Gesundheit damit zusammenhängt**.

Die Karte ist ein Rechteck mit Wänden und einem grünen, säurehaltigen Boden, der den Spieler regelmäßig **verletzt**. Anfangs gibt es einige Medkits, die gleichmäßig über die Karte verteilt sind. Ab und zu fällt ein neues Medkit vom Himmel. **Medkits heilen einen Teil der Gesundheit des Spielers** - um zu überleben, muss der Agent sie aufsammeln. Die Episode endet nach dem Tod des Spielers oder bei Zeitüberschreitung.

Weitere Konfiguration:
- Lebendige_Belohnung = 1
- 3 verfügbare Tasten: links drehen, rechts drehen, vorwärts gehen
- 1 verfügbare Spielvariable: GESUNDHEIT
- Todesstrafe = 100

Weitere Informationen über die in ViZDoom verfügbaren Szenarien finden Sie [hier](https://github.com/Farama-Foundation/ViZDoom/tree/master/scenarios).

Es gibt auch eine Reihe von komplexeren Szenarien, die für ViZDoom erstellt wurden, wie z. B. die auf [dieser github-Seite](https://github.com/edbeeching/3d_control_deep_rl) beschriebenen.



## Training des Agenten
- Wir werden den Agenten für 4000000 Schritte trainieren, was etwa 20 Minuten dauern wird.

In [ ]:
## Start the training, this should take around 15 minutes
register_vizdoom_components()

# The scenario we train on today is health gathering
# other scenarios include "doom_basic", "doom_two_colors_easy", "doom_dm", "doom_dwango5", "doom_my_way_home", "doom_deadly_corridor", "doom_defend_the_center", "doom_defend_the_line"
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=8", "--num_envs_per_worker=4", "--train_for_env_steps=4000000"])

status = run_rl(cfg)

## Schauen wir uns die Leistung der trainierten Richtlinie an und geben ein Video des Agenten aus.

In [ ]:
from sample_factory.enjoy import enjoy
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10"], evaluation=True)
status = enjoy(cfg)

## Nun wollen wir die Leistung des Agenten visualisieren

In [ ]:
from base64 import b64encode
from IPython.display import HTML

mp4 = open('/content/train_dir/default_experiment/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Der Agent hat etwas gelernt, aber seine Leistung könnte besser sein. Wir müssten eindeutig länger trainieren. Aber lassen Sie uns dieses Modell in den Hub hochladen.

## Jetzt können wir deinen Checkpoint und dein Video in den Hugging Face Hub hochladen.




Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">

- Kopieren Sie das Token
- Führen Sie die Zelle unten aus und fügen Sie das Token ein

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login`

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [ ]:
from sample_factory.enjoy import enjoy

hf_username = "ThomasSimonini" # insert your HuggingFace username here

cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--max_num_frames=100000", "--push_to_hub", f"--hf_repository={hf_username}/rl_course_vizdoom_health_gathering_supreme"], evaluation=True)
status = enjoy(cfg)

## Lassen Sie uns ein anderes Modell laden




Die Leistung dieses Agenten war gut, kann aber noch besser werden! Laden wir einen Agenten, der für 10B Zeitschritte trainiert wurde, vom Hub herunter und visualisieren ihn.

In [ ]:
#download the agent from the hub
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_health_gathering_supreme_2222 -d ./train_dir


In [ ]:
!ls train_dir/doom_health_gathering_supreme_2222

In [ ]:
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--experiment=doom_health_gathering_supreme_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)

In [ ]:
mp4 = open('/content/train_dir/doom_health_gathering_supreme_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Einige zusätzliche Herausforderungen 🏆: Doom Deathmatch

Einen Agenten für ein Doom-Deathmatch zu trainieren **dauert viele Stunden auf einem leistungsfähigeren Rechner, als er in Colab verfügbar ist**.

Glücklicherweise haben wir **bereits einen Agenten in diesem Szenario trainiert und er ist im 🤗 Hub verfügbar!** Laden wir das Modell herunter und visualisieren wir die Leistung des Agenten.

In [ ]:
# Download the agent from the hub
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_deathmatch_bots_2222 -d ./train_dir

Wenn der Agent sehr lange spielt, kann die Videoerstellung **10 Minuten** dauern.

In [ ]:

from sample_factory.enjoy import enjoy
register_vizdoom_components()
env = "doom_deathmatch_bots"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=1", "--experiment=doom_deathmatch_bots_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)
mp4 = open('/content/train_dir/doom_deathmatch_bots_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


Sie **können versuchen, Ihren Agenten in dieser Umgebung** zu trainieren, indem Sie den obigen Code verwenden, aber nicht auf colab.
**Viel Glück 🤞**

Wenn Sie ein einfacheres Szenario bevorzugen, **können Sie versuchen, in einem anderen ViZDoom-Szenario zu trainieren, z. B. `doom_deadly_corridor` oder `doom_defend_the_center`.



---


Damit ist die letzte Einheit abgeschlossen. Aber wir sind noch nicht fertig! Der folgende **Bonusabschnitt enthält einige der interessantesten, fortgeschrittensten und innovativsten Arbeiten im Bereich des Deep Reinforcement Learning**.

## Lernen Sie weiter, bleiben Sie fantastisch 🤗